# A.1 - RGB Only Training (V2)

**Experiment:** A.1 - RGB Baseline
**Input:** RGB images (3-channel)
**Objective:** Baseline detection using RGB only
**Classes:** 1 (fresh_fruit_bunch)

## Uniform Augmentation (All Experiments)
- translate: 0.1
- scale: 0.5
- fliplr: 0.5
- hsv_h: 0.0 (disabled)
- hsv_s: 0.0 (disabled)
- hsv_v: 0.0 (disabled)
- erasing: 0.0
- mosaic: 0.0
- mixup: 0.0

In [1]:
# Cell 1: Environment Setup
import os
import sys
import torch
import numpy as np
from pathlib import Path

IS_KAGGLE = os.path.exists('/kaggle/input')

if IS_KAGGLE:
    BASE_PATH = Path('/kaggle/working')
    DATASET_PATH = Path('/kaggle/input/ffb-localization-dataset/ffb_localization')
else:
    BASE_PATH = Path('D:/Work/Assisten Dosen/Anylabel/Experiments')
    DATASET_PATH = BASE_PATH / 'datasets' / 'ffb_localization'

RUNS_PATH = BASE_PATH / 'runs' / 'detect'
KAGGLE_OUTPUT = BASE_PATH / 'kaggleoutput'
KAGGLE_OUTPUT.mkdir(parents=True, exist_ok=True)

print(f'Running on: {"Kaggle" if IS_KAGGLE else "Local"}')
print(f'CUDA: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

Running on: Kaggle
CUDA: True
GPU: Tesla T4


In [2]:
# Cell 2: Install
!pip install -q ultralytics
from ultralytics import YOLO
import pandas as pd
import json
from datetime import datetime
import shutil
import gc
import time

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.6 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# Cell 3: Configuration
AUGMENT_PARAMS = {
    'translate': 0.1,
    'scale': 0.5,
    'fliplr': 0.5,
    'hsv_h': 0.0,
    'hsv_s': 0.0,
    'hsv_v': 0.0,
    'erasing': 0.0,
    'mosaic': 0.0,
    'mixup': 0.0,
    'degrees': 0.0,
    'copy_paste': 0.0,
}

SEEDS = [42, 123, 456, 789, 101]
EXP_PREFIX = 'exp_a1_rgb_v2'
EPOCHS = 100
PATIENCE = 30
IMGSZ = 640
BATCH_SIZE = 16
DEVICE = 0 if torch.cuda.is_available() else 'cpu'

print(f'Experiment: A.1 RGB Only (V2)')
print(f'Seeds: {SEEDS}')
print(f'Epochs: {EPOCHS}')

Experiment: A.1 RGB Only (V2)
Seeds: [42, 123, 456, 789, 101]
Epochs: 100


In [4]:
# Cell 4: Dataset Verification
for split in ['train', 'val', 'test']:
    img_dir = DATASET_PATH / 'images' / split
    lbl_dir = DATASET_PATH / 'labels' / split
    imgs = len(list(img_dir.glob('*.png'))) if img_dir.exists() else 0
    lbls = len(list(lbl_dir.glob('*.txt'))) if lbl_dir.exists() else 0
    print(f'{split}: {imgs} images, {lbls} labels')

yaml_content = f"""
path: {DATASET_PATH}
train: images/train
val: images/val
test: images/test
nc: 1
names: ['fresh_fruit_bunch']
"""

config_path = BASE_PATH / 'dataset_rgb_v2.yaml'
with open(config_path, 'w') as f:
    f.write(yaml_content)
print(f'Config: {config_path}')

train: 280 images, 280 labels
val: 80 images, 80 labels
test: 40 images, 40 labels
Config: /kaggle/working/dataset_rgb_v2.yaml


In [5]:
# Cell 5: Training Loop
results_all = {}

for idx, seed in enumerate(SEEDS, 1):
    print(f'\n=== Training Seed {seed} ({idx}/{len(SEEDS)}) ===')
    
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    
    model = YOLO('yolo11n.pt')
    
    try:
        results = model.train(
            data=str(config_path),
            epochs=EPOCHS,
            patience=PATIENCE,
            seed=seed,
            name=f'{EXP_PREFIX}_seed{seed}',
            project=str(RUNS_PATH),
            exist_ok=True,
            imgsz=IMGSZ,
            batch=BATCH_SIZE,
            device=DEVICE,
            **AUGMENT_PARAMS,
        )
        results_all[seed] = {'completed': True}
        print(f'Seed {seed} done!')
    except Exception as e:
        print(f'Seed {seed} failed: {e}')
        results_all[seed] = {'error': str(e)}
    
    del model
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()


=== Training Seed 42 (1/5) ===
Ultralytics 8.4.8 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/dataset_rgb_v2.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=100, erasing=0.0, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=0.0, multi_scale=0.0, name=exp_a1_rgb_v2_seed42, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto

In [6]:
# Cell 6: Evaluation
results_dict = {}

for seed in SEEDS:
    model_path = RUNS_PATH / f'{EXP_PREFIX}_seed{seed}' / 'weights' / 'best.pt'
    if not model_path.exists():
        continue
    
    model = YOLO(str(model_path))
    metrics = model.val(data=str(config_path), split='test', device=DEVICE)
    
    results_dict[seed] = {
        'mAP50': metrics.box.map50,
        'mAP50-95': metrics.box.map,
        'Precision': metrics.box.mp,
        'Recall': metrics.box.mr,
    }
    print(f'Seed {seed}: mAP50={metrics.box.map50:.4f}')
    
    del model
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

Ultralytics 8.4.8 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 101 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 1.1±1.4 ms, read: 70.3±22.6 MB/s, size: 2705.1 KB)
val: Scanning /kaggle/input/ffb-localization-dataset/ffb_localization/labels/test... 40 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 40/40 52.4it/s 0.8s
WARNING ⚠️ val: Cache directory /kaggle/input/ffb-localization-dataset/ffb_localization/labels is not writable, cache not saved.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 1.8it/s 1.7s
                   all         40        105      0.757      0.831      0.881      0.366
Speed: 3.4ms preprocess, 13.6ms inference, 0.0ms loss, 7.3ms postprocess per image
Results saved to /kaggle/working/runs/detect/val
Seed 42: mAP50=0.8809
Ultralytics 8.4.8 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary

In [7]:
# Cell 7: Summary
if results_dict:
    df = pd.DataFrame(results_dict).T
    df.index.name = 'Seed'
    
    print('A.1 RGB ONLY (V2) - RESULTS\n')
    print(df.to_string())
    
    print(f'\nMean:')
    for col in df.columns:
        print(f'  {col}: {df[col].mean():.4f} ± {df[col].std():.4f}')
    
    print(f'\nBest Seed: {df["mAP50"].idxmax()}')

A.1 RGB ONLY (V2) - RESULTS

         mAP50  mAP50-95  Precision    Recall
Seed                                         
42    0.880852  0.366196   0.757114  0.831260
123   0.832373  0.369654   0.868399  0.691322
456   0.814832  0.365704   0.771671  0.804699
789   0.832504  0.375082   0.829569  0.741720
101   0.831693  0.345793   0.787321  0.733333

Mean:
  mAP50: 0.8385 ± 0.0249
  mAP50-95: 0.3645 ± 0.0111
  Precision: 0.8028 ± 0.0456
  Recall: 0.7605 ± 0.0566

Best Seed: 42


In [8]:
# Cell 8: Save Results
output_file = KAGGLE_OUTPUT / 'a1_rgb_v2_results.txt'

with open(output_file, 'w') as f:
    f.write('A.1 RGB Only (V2) Results\n')
    f.write(f'Generated: {datetime.now()}\n\n')
    if results_dict:
        f.write(df.to_string())
        f.write('\n\nSummary:\n')
        for col in df.columns:
            f.write(f'{col}: {df[col].mean():.4f} ± {df[col].std():.4f}\n')

print(f'Results saved: {output_file}')

Results saved: /kaggle/working/kaggleoutput/a1_rgb_v2_results.txt


In [9]:
# Cell 9: Create Archives
if RUNS_PATH.exists():
    shutil.make_archive(str(BASE_PATH / 'a1_rgb_v2_runs'), 'zip', RUNS_PATH)
    print('✓ a1_rgb_v2_runs.zip created')

shutil.make_archive(str(BASE_PATH / 'a1_rgb_v2_output'), 'zip', KAGGLE_OUTPUT)
print('✓ a1_rgb_v2_output.zip created')

✓ a1_rgb_v2_runs.zip created
✓ a1_rgb_v2_output.zip created
